<a href="https://colab.research.google.com/github/123shwetarohokale/563-ShwetaR/blob/main/Robotics_Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

def skew_symmetric(omega):
    """Returns the 3x3 skew-symmetric matrix of a vector."""
    wx, wy, wz = omega
    return np.array([
        [ 0, -wz,  wy],
        [ wz,  0, -wx],
        [-wy, wx,   0]
    ])

def omega_matrix(omega):
    """Constructs the 4x4 Omega(omega) matrix for quaternion kinematics."""
    omega_cross = skew_symmetric(omega)
    top = np.hstack(([0], -omega))
    bottom = np.hstack((omega[:, np.newaxis], -omega_cross))
    return np.vstack((top[np.newaxis, :], bottom))

def closed_form_quaternion_update(q, omega, dt):
    """
    Updates quaternion using closed-form exponential of Omega matrix.

    Parameters:
    q     -- current quaternion [q0, q1, q2, q3] (scalar-first)
    omega -- angular velocity vector [wx, wy, wz] in rad/s
    dt    -- time interval in seconds

    Returns:
    q_next -- updated quaternion (normalized)
    """
    norm_omega = np.linalg.norm(omega)
    if norm_omega == 0:
        return q

    theta = norm_omega * dt
    Omega = 0.5 * omega_matrix(omega)
    I = np.eye(4)

    exp_Omega_dt = (
        np.cos(theta / 2) * I +
        (np.sin(theta / 2) / norm_omega) * Omega
    )

    q_next = exp_Omega_dt @ q
    return q_next / np.linalg.norm(q_next)

def run_simulation():
    # Initial quaternion (no rotation)
    q = np.array([1.0, 0.0, 0.0, 0.0])
    q_initial = q.copy()

    # Time step
    dt = 0.25  # seconds

    # Define the sequence of angular velocities
    rotations = [
        ("Rotate +π rad/s about X",  np.array([ np.pi, 0.0, 0.0])),
        ("Rotate +π rad/s about Z",  np.array([0.0, 0.0,  np.pi])),
        ("Rotate −π rad/s about X",  np.array([-np.pi, 0.0, 0.0])),
        ("Rotate −π rad/s about Z",  np.array([0.0, 0.0, -np.pi]))
    ]

    print("Initial quaternion q(0):", q)

    # Apply each rotation
    for step_name, omega in rotations:
        q = closed_form_quaternion_update(q, omega, dt)
        print(f"After '{step_name}': q = {np.round(q, decimals=6)}")

    # Final result
    print("\nFinal quaternion q(1):", np.round(q, 6))
    print("Initial quaternion q(0):", q_initial)

    # Check if final quaternion is approximately equal to the initial
    if np.allclose(q, q_initial, atol=1e-6) or np.allclose(q, -q_initial, atol=1e-6):
        print("Result:Final orientation is the same as the initial orientation (modulo sign).")
    else:
        print("Result:Final orientation is different from the initial orientation.")

if __name__ == "__main__":
    run_simulation()


Initial quaternion q(0): [1. 0. 0. 0.]
After 'Rotate +π rad/s about X': q = [0.97922  0.202803 0.       0.      ]
After 'Rotate +π rad/s about Z': q = [ 0.958871  0.198589 -0.041129  0.198589]
After 'Rotate −π rad/s about X': q = [ 0.97922   0.       -0.080549  0.186121]
After 'Rotate −π rad/s about Z': q = [ 0.996617  0.016336 -0.078875 -0.016336]

Final quaternion q(1): [ 0.996617  0.016336 -0.078875 -0.016336]
Initial quaternion q(0): [1. 0. 0. 0.]
Result:Final orientation is different from the initial orientation.


In [3]:
import numpy as np
import csv

def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm > 0 else v

def skew(v):
    """Return the skew-symmetric matrix of vector v."""
    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def quat_to_rotmat(q):
    """Convert unit quaternion to rotation matrix."""
    q0, q1, q2, q3 = q
    return np.array([
        [1 - 2*(q2**2 + q3**2),     2*(q1*q2 - q0*q3),     2*(q1*q3 + q0*q2)],
        [    2*(q1*q2 + q0*q3), 1 - 2*(q1**2 + q3**2),     2*(q2*q3 - q0*q1)],
        [    2*(q1*q3 - q0*q2),     2*(q2*q3 + q0*q1), 1 - 2*(q1**2 + q2**2)]
    ])

def quat_mult(q1, q2):
    """Quaternion multiplication q1 ◦ q2."""
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    return np.array([
        w1*w2 - x1*x2 - y1*y2 - z1*z2,
        w1*x2 + x1*w2 + y1*z2 - z1*y2,
        w1*y2 - x1*z2 + y1*w2 + z1*x2,
        w1*z2 + x1*y2 - y1*x2 + z1*w2
    ])

def update_mahony_filter(q, b, gyro, acc, mag, dt, kp, ki, ka, km):
    # Normalize inputs
    acc = normalize(acc)
    mag = normalize(mag)

    R = quat_to_rotmat(q)
    v_hat_a = R.T @ np.array([0, 0, 1])
    v_hat_m = R.T @ np.array([1, 0, 0])

    omega_mes = ka * np.cross(acc, v_hat_a) + km * np.cross(mag, v_hat_m)

    u = gyro - b + kp * omega_mes
    dq = 0.5 * quat_mult(q, np.array([0, *u]))  # q ◦ p(u)
    q_next = q + dq * dt
    q_next /= np.linalg.norm(q_next)

    db = -ki * omega_mes * dt
    b_next = b + db

    return q_next, b_next

def run_mahony_filter(input_file, dt=0.01, kp=2.0, ki=0.05, ka=1.0, km=1.0):
    with open(input_file, 'r') as f:
        reader = csv.reader(f)
        data = []
        for row in reader:
            if len(row) < 10: continue
            try:
                t = float(row[0])
                gyro = np.array([float(row[4]), float(row[5]), float(row[6])])
                acc = np.array([float(row[7]), float(row[8]), float(row[9])])
                mag = np.array([float(row[1]), float(row[2]), float(row[3])])
                data.append((t, gyro, acc, mag))
            except: continue

    q = np.array([1.0, 0.0, 0.0, 0.0])
    b = np.zeros(3)
    orientation = [(0.0, q.copy())]

    for t, gyro, acc, mag in data:
        q, b = update_mahony_filter(q, b, gyro, acc, mag, dt, kp, ki, ka, km)
        orientation.append((t, q.copy()))

    return orientation

def save_output(orientation, filename='mahony_output.csv'):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['time', 'q0', 'q1', 'q2', 'q3'])
        for t, q in orientation:
            writer.writerow([t] + q.tolist())

if __name__ == "__main__":
    input_file = 'input.csv'
    orientation = run_mahony_filter(input_file)
    save_output(orientation)
    print("✅ Mahony filter applied and orientation saved.")


FileNotFoundError: [Errno 2] No such file or directory: 'input.csv'